In [14]:
import optionsdata as od
from datetime import datetime
failed = await od.fetch_options(["ZEB"], datetime(2024, 1, 1).date(), datetime(2025, 2, 18).date())
print(failed)

[]


In [11]:
import pandas as pd
df = pd.read_parquet('data/2024_TRI.parquet')
df[df["volume"] > 0]

,expiration,strike,option_type,underlying_symbol,underlying_price,contract_symbol,eod_date,dte,open_interest,volume,...,close_bid_size,close_ask,close_ask_size,prev_close,change,implied_volatility,transactions,total_value,settlement_price,Breakeven
query_date,,,,,,,,,,,,,,,,,,,,,
2024-01-01,2024-02-16,185.0,call,TRI:CA,191.95,TRI 240216C185.00,2024-01-02,45,4,4,...,10,10.65,25,12.40,-1.90,0.20152,1,42.0,10.65,195.65
2024-01-01,2024-02-16,195.0,call,TRI:CA,191.95,TRI 240216C195.00,2024-01-02,45,5,5,...,35,4.05,25,4.90,-1.40,0.17577,1,17.0,4.05,199.05
2024-01-01,2024-03-15,200.0,call,TRI:CA,191.95,TRI 240315C200.00,2024-01-02,73,16,2,...,45,3.25,10,3.95,-1.25,0.16249,1,5.0,3.25,203.25
2024-01-01,2024-03-15,205.0,call,TRI:CA,191.95,TRI 240315C205.00,2024-01-02,73,21,1,...,55,1.80,20,2.30,-0.75,0.15905,1,1.0,1.80,206.80
2024-01-02,2024-02-16,185.0,call,TRI:CA,191.95,TRI 240216C185.00,2024-01-02,45,4,4,...,10,10.65,25,12.40,-1.90,0.20152,1,42.0,10.65,195.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-18,2025-03-21,230.0,put,TRI:CA,246.35,TRI 250321P230.00,2025-02-18,31,2,4,...,30,1.50,30,1.70,-0.05,0.23292,2,6.0,1.45,228.55
2025-02-18,2025-03-21,235.0,put,TRI:CA,246.35,TRI 250321P235.00,2025-02-18,31,5,2,...,20,2.25,20,2.50,-0.35,0.22716,1,4.0,2.25,232.75
2025-02-18,2025-03-21,260.0,call,TRI:CA,246.35,TRI 250321C260.00,2025-02-18,31,29,1,...,30,1.30,20,1.55,-0.55,0.17630,1,1.0,1.30,261.30
